# Análise Titanic - Feature Scaling

Neste notebook iremos continuar com a análise de dados dos passageiros do Titanic. Agora iremos trabalhar com os dados com alguns tratamentos possíveis


## Etapa inicial - Baixar os dados

In [1]:
import pandas as pd

dados = pd.read_excel('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls')

dados.head(3)
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
pclass       1309 non-null int64
survived     1309 non-null int64
name         1309 non-null object
sex          1309 non-null object
age          1046 non-null float64
sibsp        1309 non-null int64
parch        1309 non-null int64
ticket       1309 non-null object
fare         1308 non-null float64
cabin        295 non-null object
embarked     1307 non-null object
boat         486 non-null object
body         121 non-null float64
home.dest    745 non-null object
dtypes: float64(3), int64(4), object(7)
memory usage: 143.2+ KB


In [3]:
## Preencher idades faltantes
dados.isna().sum()

## Inserir Idades faltantes com a Mediana
dados['age'].fillna(dados['age'].median(), inplace = True)

## Criar variável "familia" como a soma entre 'sibsp' e 'parch'
dados['familia'] = dados['sibsp'] + dados['parch'] 

## Transformar variáveis categóricas
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labelencoder.fit_transform(dados['sex'])

dados_ajustados2 = pd.get_dummies(dados, columns=["pclass","sex"])

del dados_ajustados2['sex_female']
del dados_ajustados2['pclass_1']

resultados_acuracia2 = pd.DataFrame(columns = ['modelo', 'acuracia'])

#### Padronizar as variáveis quantitativas
from sklearn import preprocessing

std_scale = preprocessing.StandardScaler().fit(dados_ajustados2[['age', 'familia']])
dados_ajustados2[['age', 'familia']] = std_scale.transform(dados_ajustados2[['age','familia']])


#######################################################
#
#               Rodar Modelos
#
#######################################################

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

modelos = [
    [LogisticRegression(fit_intercept = True),'lr_sklearn'],
    [KNeighborsClassifier(n_neighbors=3),'knn'],
    [GaussianNB(),'naive_bayes'],
    [LinearDiscriminantAnalysis(),'lda'],
    [svm.SVC(gamma='scale'),'svm'],
    [DecisionTreeClassifier(), 'arvore_decisao']
]

# Criar 
def analise(modelo, nome, dados, tabela_acuracia):
    
    modelo.fit(dados[['pclass_2','pclass_3','age','sex_male','familia']],dados['survived'])
    previsto = modelo.predict(dados[['pclass_2','pclass_3','age','sex_male','familia']])
    acuracia = accuracy_score(dados['survived'],previsto)
    tabela_acuracia = tabela_acuracia.append({'modelo':nome, 'acuracia':acuracia}, ignore_index = True)
                                             
    return tabela_acuracia

In [6]:
for item in modelos:
    resultados_acuracia = analise(item[0],item[1], dados_ajustados2, resultados_acuracia2)
    resultados_acuracia2 = resultados_acuracia

TypeError: must be real number, not str